In [1]:
from sklearn import datasets
import numpy as np
from chainer import cuda, Function, FunctionSet, gradient_check, Variable, optimizers
from chainer import computational_graph as c
import chainer.functions as F
from chainer import optimizers
import six

In [2]:
iris = datasets.load_iris()

In [3]:
X = iris.data

In [4]:
X

array([[ 5.1,  3.5,  1.4,  0.2],
       [ 4.9,  3. ,  1.4,  0.2],
       [ 4.7,  3.2,  1.3,  0.2],
       [ 4.6,  3.1,  1.5,  0.2],
       [ 5. ,  3.6,  1.4,  0.2],
       [ 5.4,  3.9,  1.7,  0.4],
       [ 4.6,  3.4,  1.4,  0.3],
       [ 5. ,  3.4,  1.5,  0.2],
       [ 4.4,  2.9,  1.4,  0.2],
       [ 4.9,  3.1,  1.5,  0.1],
       [ 5.4,  3.7,  1.5,  0.2],
       [ 4.8,  3.4,  1.6,  0.2],
       [ 4.8,  3. ,  1.4,  0.1],
       [ 4.3,  3. ,  1.1,  0.1],
       [ 5.8,  4. ,  1.2,  0.2],
       [ 5.7,  4.4,  1.5,  0.4],
       [ 5.4,  3.9,  1.3,  0.4],
       [ 5.1,  3.5,  1.4,  0.3],
       [ 5.7,  3.8,  1.7,  0.3],
       [ 5.1,  3.8,  1.5,  0.3],
       [ 5.4,  3.4,  1.7,  0.2],
       [ 5.1,  3.7,  1.5,  0.4],
       [ 4.6,  3.6,  1. ,  0.2],
       [ 5.1,  3.3,  1.7,  0.5],
       [ 4.8,  3.4,  1.9,  0.2],
       [ 5. ,  3. ,  1.6,  0.2],
       [ 5. ,  3.4,  1.6,  0.4],
       [ 5.2,  3.5,  1.5,  0.2],
       [ 5.2,  3.4,  1.4,  0.2],
       [ 4.7,  3.2,  1.6,  0.2],
       [ 4

In [5]:
Y = iris.target

In [6]:
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [8]:
X = X.astype(np.float32)
X

array([[ 5.0999999 ,  3.5       ,  1.39999998,  0.2       ],
       [ 4.9000001 ,  3.        ,  1.39999998,  0.2       ],
       [ 4.69999981,  3.20000005,  1.29999995,  0.2       ],
       [ 4.5999999 ,  3.0999999 ,  1.5       ,  0.2       ],
       [ 5.        ,  3.5999999 ,  1.39999998,  0.2       ],
       [ 5.4000001 ,  3.9000001 ,  1.70000005,  0.40000001],
       [ 4.5999999 ,  3.4000001 ,  1.39999998,  0.30000001],
       [ 5.        ,  3.4000001 ,  1.5       ,  0.2       ],
       [ 4.4000001 ,  2.9000001 ,  1.39999998,  0.2       ],
       [ 4.9000001 ,  3.0999999 ,  1.5       ,  0.1       ],
       [ 5.4000001 ,  3.70000005,  1.5       ,  0.2       ],
       [ 4.80000019,  3.4000001 ,  1.60000002,  0.2       ],
       [ 4.80000019,  3.        ,  1.39999998,  0.1       ],
       [ 4.30000019,  3.        ,  1.10000002,  0.1       ],
       [ 5.80000019,  4.        ,  1.20000005,  0.2       ],
       [ 5.69999981,  4.4000001 ,  1.5       ,  0.40000001],
       [ 5.4000001 ,  3.

In [9]:
Y = Y.flatten().astype(np.int32)
Y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [10]:
#入力層4,中間層10,出力層3のNN
n_units   = 10
model = FunctionSet(l1=F.Linear(4, n_units),
                    l2=F.Linear(n_units, n_units),
                    l3=F.Linear(n_units, 3))

In [11]:
#FeedForwardの関数定義
def forward(x_data, y_data, train=True):
    x, t = Variable(x_data), Variable(y_data)
    h1 = F.dropout(F.relu(model.l1(x)),  train=train)
    h2 = F.dropout(F.relu(model.l2(h1)), train=train)
    y  = model.l3(h2)
    return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

In [12]:
x_train, x_test = np.split(X,[75])

In [13]:
y_train, y_test = np.split(Y,[75])

In [14]:
N_test = y_test.size

In [16]:
optimizer = optimizers.SGD()
optimizer.setup(model.collect_parameters())
optimizer.zero_grads()

In [17]:
N = 75
batchsize = 5
n_epoch = 1000
n_units = 1000

In [18]:
for epoch in six.moves.range(1, n_epoch + 1):
    print('epoch', epoch)

    # training
    perm = np.random.permutation(N)
    sum_accuracy = 0
    sum_loss = 0
    for i in six.moves.range(0, N, batchsize):
        x_batch = x_train[perm[i:i + batchsize]]
        y_batch = y_train[perm[i:i + batchsize]]

        optimizer.zero_grads()
        loss, acc = forward(x_batch, y_batch)
        loss.backward()
        optimizer.update()

        if epoch == 1 and i == 0:
            with open("graph.dot", "w") as o:
                o.write(c.build_computational_graph((loss, )).dump())
            with open("graph.wo_split.dot", "w") as o:
                g = c.build_computational_graph((loss, ),
                                                remove_split=True)
                o.write(g.dump())
            print('graph generated')

        sum_loss += float(cuda.to_cpu(loss.data)) * len(y_batch)
        sum_accuracy += float(cuda.to_cpu(acc.data)) * len(y_batch)

    print('train mean loss={}, accuracy={}'.format(
        sum_loss / N, sum_accuracy / N))

    # evaluation
    sum_accuracy = 0
    sum_loss = 0
    for i in six.moves.range(0, N_test, batchsize):
        x_batch = x_test[i:i + batchsize]
        y_batch = y_test[i:i + batchsize]

        loss, acc = forward(x_batch, y_batch, train=False)

        sum_loss += float(cuda.to_cpu(loss.data)) * len(y_batch)
        sum_accuracy += float(cuda.to_cpu(acc.data)) * len(y_batch)

    print('test  mean loss={}, accuracy={}'.format(
        sum_loss / N_test, sum_accuracy / N_test))

('epoch', 1)
graph generated
train mean loss=1.23494615952, accuracy=0.560000012318
test  mean loss=2.75743788481, accuracy=0.0
('epoch', 2)
train mean loss=0.983792092403, accuracy=0.57333334585
test  mean loss=2.93676792383, accuracy=0.0
('epoch', 3)
train mean loss=0.831130123138, accuracy=0.626666683952
test  mean loss=2.8634199063, accuracy=0.293333335718
('epoch', 4)
train mean loss=0.870888139804, accuracy=0.600000013908
test  mean loss=2.8458726565, accuracy=0.0
('epoch', 5)
train mean loss=0.865969447295, accuracy=0.586666684349
test  mean loss=2.81197864612, accuracy=0.0
('epoch', 6)
train mean loss=0.872250501315, accuracy=0.600000013908
test  mean loss=2.93759588401, accuracy=0.0
('epoch', 7)
train mean loss=0.842373959223, accuracy=0.666666681568
test  mean loss=2.79457238913, accuracy=0.333333333333
('epoch', 8)
train mean loss=0.834051879247, accuracy=0.586666679382
test  mean loss=2.74117934704, accuracy=0.333333333333
('epoch', 9)
train mean loss=0.80164886713, accurac

In [19]:
#FeedForwardの関数定義
def predict_forward(x_data):
    x = Variable(x_data)
    h1 = F.relu(model.l1(x))
    h2 = F.relu(model.l2(h1))
    y  = model.l3(h2)
    return y

In [20]:
test_x = np.array([[6 ,  4 ,  1.5,  0.2 ]],dtype=np.float32)

In [22]:
y = predict_forward(test_x)

In [23]:
F.softmax(y).data

array([[  9.99953032e-01,   3.33785392e-05,   1.35438913e-05]], dtype=float32)